# Learning Scala - Day 4

Today I'm going to practice some advanced Scala concepts:
- For loops with yield
- Functions and predicates
- Trying out partial functions
- Composing functions together
- Understanding blocks
- Looking at blocking/non-blocking operations
- Working with private methods
- Error handling
- Async operations with Future
- Implicit conversions (if I have time)

## 1. Playing with yield

Let's see how yield works with for loops. I think it's like list comprehension in Python?

In [ ]:
// First, let's try a simple list
val nums = List(1,2,3,4,5)

// Double each number
val doubled = for(n <- nums) yield n * 2
println(s"Doubled: $doubled")

// Only double even numbers
val doubledEvens = for {
  n <- nums
  if n % 2 == 0  // Filter for even numbers
} yield n * 2
println(s"Doubled evens: $doubledEvens")

// Let's try with Option too
def divide(x: Int, y: Int): Option[Int] = 
  if (y == 0) None else Some(x/y)

// This is cool - chaining Options with for
val result = for {
  a <- divide(10, 2)    // Should be Some(5)
  b <- divide(a, 1)     // Should be Some(5)
} yield b + 1

println(s"Result after divisions: $result")

// Let me try to make my own iterator
// Want to generate first N fibonacci numbers
class FibNumbers(n: Int) extends Iterator[Int] {
  private var (a, b) = (0, 1)
  private var count = 0
  
  def hasNext = count < n
  
  def next() = {
    val result = a
    a = b
    b = result + b
    count += 1
    result
  }
}

// Use it with for/yield
val fibs = for(x <- new FibNumbers(6)) yield x
println("First 6 Fibonacci numbers:")
fibs.foreach(println)

// Make sure everything worked
assert(doubled == List(2,4,6,8,10))
assert(doubledEvens == List(4,8))
assert(result == Some(6))

Original numbers: List(1, 2, 3, 4, 5)
Doubled even numbers: List(4, 8)
Result of division chain: Some(10)
First 6 Fibonacci numbers:
Doubled even numbers: List(4, 8)
Result of division chain: Some(10)
First 6 Fibonacci numbers:
0
1
1
2
0
1
1
2
3
5
3
5


numbers: List[Int] = List(1, 2, 3, 4, 5)
doubled: List[Int] = List(4, 8)
defined function divide
result: Option[Int] = Some(value = 10)
defined class FibonacciIterator
fibNumbers: Iterator[Int] = empty iterator

## 2. Working with Predicates

These are just functions that return true/false. Let's try to combine them in different ways.

In [ ]:
// Some basic checks
def isEven(n: Int) = n % 2 == 0
def isPositive(n: Int) = n > 0
def isSmall(n: Int) = n < 10

// Test them out
val numbers = List(-5, -2, 0, 3, 6, 9, 12)

println("Even numbers:")
numbers.filter(isEven).foreach(println)

println("\nPositive numbers:")
numbers.filter(isPositive).foreach(println)

// Can I combine these? Let's try making AND and OR functions
def combine(f1: Int => Boolean, f2: Int => Boolean, useAnd: Boolean): Int => Boolean = {
  n => if(useAnd) f1(n) && f2(n) else f1(n) || f2(n)
}

// Test combinations
val evenAndPositive = combine(isEven, isPositive, true)
println("\nEven AND positive:")
numbers.filter(evenAndPositive).foreach(println)

val evenOrSmall = combine(isEven, isSmall, false)
println("\nEven OR small:")
numbers.filter(evenOrSmall).foreach(println)

// Maybe I can make it more generic with type parameters
def and[A](p1: A => Boolean, p2: A => Boolean): A => Boolean = 
  a => p1(a) && p2(a)

def or[A](p1: A => Boolean, p2: A => Boolean): A => Boolean = 
  a => p1(a) || p2(a)

// This is nicer!
val betterCombination = and(isEven, isPositive)
println("\nBetter way to combine predicates:")
numbers.filter(betterCombination).foreach(println)

// Quick checks
assert(numbers.exists(isEven), "Should have some even numbers")
assert(numbers.exists(isPositive), "Should have some positive numbers")
assert(numbers.filter(evenAndPositive).forall(n => isEven(n) && isPositive(n)), 
       "Combined predicate should work")

Even numbers:
List(-2, 0, 6, 12)

Positive numbers:
List(3, 6, 9, 12)

Even and positive numbers:
List(6, 12)

Numbers that are even or less than 10:
List(-2, 0, 6, 12)

Positive numbers:
List(3, 6, 9, 12)

Even and positive numbers:
List(6, 12)

Numbers that are even or less than 10:
List(-5, -2, 0, 3, 6, 9, 12)

Odd numbers:
List(-5, 3, 9)
List(-5, -2, 0, 3, 6, 9, 12)

Odd numbers:
List(-5, 3, 9)


isEven: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3213/0x000000030196eb00@52bb118f
isPositive: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3214/0x000000030196eee8@ce31fbf
isLessThan10: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3215/0x000000030196f2d0@53a3cc79
defined function and
defined function or
defined function not
numbers: List[Int] = List(-5, -2, 0, 3, 6, 9, 12)
isEvenAndPositive: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3216/0x000000030196f6b8@4b94b35
isEvenOrLessThan10: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3217/0x000000030196fa88@f83975b
isOdd: Int => Boolean = ammonite.$sess.cmd2$Helper$$Lambda$3218/0x0000000301982000@2fb8c2f
testNumbers: List[Int] = List(2, 4, 6, 8)

## 3. Partial Applications

Partial application allows you to fix some parameters of a function, creating a new function with fewer parameters. This is a powerful technique for function reuse and specialization.

In [3]:
// Basic function with multiple parameters
def formatMessage(template: String, name: String, value: Int): String = 
  template.format(name, value)

// Partial application with placeholder syntax
val greetingTemplate = formatMessage("Hello %s, your score is %d")
println(greetingTemplate("Alice", 95))
println(greetingTemplate("Bob", 87))

// Curried function definition
def multiply(x: Int)(y: Int): Int = x * y

// Creating specialized functions through partial application
val double = multiply(2)
val triple = multiply(3)

println(s"Double of 5: ${double(5)}")
println(s"Triple of 5: ${triple(5)}")

// Practical example: Configuration with partial application
case class Config(host: String, port: Int)

def connectToDatabase(config: Config)(database: String)(query: String): String = 
  s"Connecting to ${config.host}:${config.port}/$database to execute: $query"

// Create partially applied function with fixed configuration
val localConfig = Config("localhost", 5432)
val connectToLocal = connectToDatabase(localConfig)

// Create database-specific connections
val connectToUsers = connectToLocal("users")
val connectToOrders = connectToLocal("orders")

// Execute queries
println(connectToUsers("SELECT * FROM users"))
println(connectToOrders("SELECT * FROM orders"))

// Assertions for validation
assert(double(5) == 10, "Double function test failed")
assert(triple(5) == 15, "Triple function test failed")
assert(connectToUsers("test").contains("localhost:5432/users"), "Database connection string test failed")

-- [E171] Type Error: cmd3.sc:5:36 ---------------------------------------------
5 |val greetingTemplate = formatMessage("Hello %s, your score is %d")
  |                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |missing argument for parameter name of method formatMessage in class Helper: (template: String, name: String, value: Int): String
Compilation FailedCompilation Failed

## 4. Function Composition

Let's try to understand how to combine functions. There are two ways: andThen and compose.
First, let's try andThen:

In [ ]:
// Let's create some simple functions first
def double(x: Int) = x * 2
def addOne(x: Int) = x + 1

// Now let's try to combine them
val result1 = double(addOne(5))  // This works but not very elegant
println(s"Traditional way: $result1")

// There must be a better way... Let's try andThen
val betterWay = double andThen addOne
println(s"Using andThen: ${betterWay(5)}")

// Cool! This is much cleaner. Let's try something more practical
case class User(name: String, age: Int)

// I want to check if a user is adult and format it nicely
def checkAge(user: User): String = 
  if (user.age >= 18) s"${user.name} is an adult"
  else s"${user.name} is a minor"

def addGreeting(msg: String): String = s"Hello! $msg"

// Let's combine them
val processUser = checkAge andThen addGreeting

// Test it out
val alice = User("Alice", 20)
val bob = User("Bob", 15)

println(processUser(alice))
println(processUser(bob))

// Make sure it works as expected
assert(processUser(alice) == "Hello! Alice is an adult")
assert(processUser(bob) == "Hello! Bob is a minor")

Double 5 and add 1: 11
Process 5: 11
Processed user: UserDTO(Alice,ADULT)
Process 5: 11
Processed user: UserDTO(Alice,ADULT)


double: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$3517/0x00000003019ce8d0@7bfee3a8
addOne: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$3518/0x00000003019cecb8@2178e1da
convertToString: Int => String = ammonite.$sess.cmd4$Helper$$Lambda$3519/0x00000003019cf0a0@89fac53
doubleAndAddOne: Int => Int = scala.Function1$$Lambda$3414/0x00000003019c83d0@4d0c1bb7
processNumber: Int => String = scala.Function1$$Lambda$3414/0x00000003019c83d0@3b918828
defined class User
defined class EnrichedUser
defined class UserDTO
checkAge: User => EnrichedUser = ammonite.$sess.cmd4$Helper$$Lambda$3520/0x00000003019cfa50@6ccfea5e
createDTO: EnrichedUser => UserDTO = ammonite.$sess.cmd4$Helper$$Lambda$3521/0x00000003019f02e8@221f23f6
processUser: User => UserDTO = scala.Function1$$Lambda$3414/0x00000003019c83d0@2d6b6057
user: User = User(name = "Alice", age = 20)

## 5. Function Composition - compose

The `compose` method is similar to `andThen` but executes functions in right-to-left order. This is more traditional in mathematical notation but can sometimes be less intuitive to read.

In [5]:
// Using the same functions as before
val double: Int => Int = _ * 2
val addOne: Int => Int = _ + 1

// Compose using compose (right-to-left)
val addOneThenDouble = double compose addOne
println(s"Add 1 to 5 then double: ${addOneThenDouble(5)}")  // (5 + 1) * 2 = 12

// Compare with andThen (left-to-right)
val doubleAndAddOne = double andThen addOne
println(s"Double 5 then add 1: ${doubleAndAddOne(5)}")      // (5 * 2) + 1 = 11

// More complex example with string processing
val removeSpaces: String => String = _.replaceAll("\\s+", "")
val countChars: String => Int = _.length
val convertToString: Int => String = _.toString

// Using compose
val processStringCompose = convertToString compose countChars compose removeSpaces
println(s"""Process "hello world" with compose: ${processStringCompose("hello world")}""")

// Using andThen (equivalent but different order)
val processStringAndThen = removeSpaces andThen countChars andThen convertToString
println(s"""Process "hello world" with andThen: ${processStringAndThen("hello world")}""")

// Assertions to verify behavior
assert(addOneThenDouble(5) == 12, "Add one then double test failed")
assert(doubleAndAddOne(5) == 11, "Double then add one test failed")
assert(processStringCompose("hello world") == "10", "String processing test failed")
assert(processStringCompose("hello world") == processStringAndThen("hello world"), 
       "Compose and andThen should yield same result")

Add 1 to 5 then double: 12
Double 5 then add 1: 11
Process "hello world" with compose: 10
Process "hello world" with andThen: 10
Double 5 then add 1: 11
Process "hello world" with compose: 10
Process "hello world" with andThen: 10


double: Int => Int = ammonite.$sess.cmd5$Helper$$Lambda$3538/0x00000003019f3620@70a3fdb2
addOne: Int => Int = ammonite.$sess.cmd5$Helper$$Lambda$3539/0x00000003019f3a08@25e2dba
addOneThenDouble: Int => Int = scala.Function1$$Lambda$3413/0x00000003019c8000@23382250
doubleAndAddOne: Int => Int = scala.Function1$$Lambda$3414/0x00000003019c83d0@517b767b
removeSpaces: String => String = ammonite.$sess.cmd5$Helper$$Lambda$3540/0x00000003019f3df0@46d2e076
countChars: String => Int = ammonite.$sess.cmd5$Helper$$Lambda$3541/0x00000003019f41b8@602ac589
convertToString: Int => String = ammonite.$sess.cmd5$Helper$$Lambda$3542/0x00000003019f4580@6153f545
processStringCompose: String => String = scala.Function1$$Lambda$3413/0x00000003019c8000@1f48426b
processStringAndThen: String => String = scala.Function1$$Lambda$3414/0x00000003019c83d0@25d15685

## 6. Blocks

Blocks in Scala are expressions enclosed in curly braces that can contain multiple statements. The last expression in a block determines its value. Blocks provide local scoping and can be used anywhere an expression is expected.

In [6]:
// Simple block returning a value
val result = {
  val a = 10
  val b = 20
  a + b  // Last expression is the block's value
}
println(s"Block result: $result")

// Block with local variables
def calculateArea(width: Int, height: Int): Int = {
  // These variables are only visible inside this block
  val w = if (width < 0) 0 else width
  val h = if (height < 0) 0 else height
  w * h
}

println(s"Area: ${calculateArea(5, 3)}")

// Nested blocks with scoping
val outer = {
  val x = 1
  val inner = {
    val y = x + 1  // Can access outer scope
    val z = y + 1
    z  // Value of inner block
  }
  inner * 2  // Value of outer block
}
println(s"Nested blocks result: $outer")

// Block as an expression in if statement
val max = if (10 > 5) {
  val diff = 10 - 5
  println(s"Difference is $diff")
  10
} else {
  5
}
println(s"Max value: $max")

// Block returning Unit
def printInfo(name: String, age: Int): Unit = {
  println("Processing information...")
  println(s"Name: $name")
  println(s"Age: $age")
  // No explicit return needed - last expression is Unit
}

printInfo("Alice", 25)

// Assertions
assert(result == 30, "Basic block calculation failed")
assert(calculateArea(5, 3) == 15, "Area calculation failed")
assert(outer == 6, "Nested block calculation failed")  // ((1 + 1 + 1) * 2)
assert(max == 10, "Block in if expression failed")

Block result: 30
Area: 15
Nested blocks result: 6
Difference is 5
Max value: 10
Processing information...
Name: Alice
Area: 15
Nested blocks result: 6
Difference is 5
Max value: 10
Processing information...
Name: Alice
Age: 25
Age: 25


result: Int = 30
defined function calculateArea
outer: Int = 6
max: Int = 10
defined function printInfo

## 7. Blocking vs Non-Blocking

Understanding the difference between blocking and non-blocking operations is crucial for building responsive applications. Let's explore both approaches and their implications.

In [7]:
import scala.concurrent.{Future, Promise, Await}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.duration._
import scala.util.{Success, Failure}

// Simulating a long-running operation
def simulateBlockingOperation(id: Int): Int = {
  println(s"Starting blocking operation $id")
  Thread.sleep(1000)  // This blocks the current thread
  println(s"Finished blocking operation $id")
  id * 2
}

// Non-blocking version using Future
def simulateNonBlockingOperation(id: Int): Future[Int] = Future {
  println(s"Starting non-blocking operation $id")
  Thread.sleep(1000)  // Still takes time, but doesn't block the calling thread
  println(s"Finished non-blocking operation $id")
  id * 2
}

// Example of blocking operations running sequentially
val startTimeBlocking = System.currentTimeMillis()
val result1 = simulateBlockingOperation(1)
val result2 = simulateBlockingOperation(2)
val timeBlocking = System.currentTimeMillis() - startTimeBlocking
println(s"Blocking operations took $timeBlocking ms")
println(s"Blocking results: $result1, $result2")

// Example of non-blocking operations running concurrently
val startTimeNonBlocking = System.currentTimeMillis()
val future1 = simulateNonBlockingOperation(1)
val future2 = simulateNonBlockingOperation(2)

// Combine futures
val combinedFuture = for {
  r1 <- future1
  r2 <- future2
} yield (r1, r2)

// We'll use Await here just for demonstration - in real code, prefer callbacks or map/flatMap
val (nonBlockingResult1, nonBlockingResult2) = 
  Await.result(combinedFuture, 2.seconds)
val timeNonBlocking = System.currentTimeMillis() - startTimeNonBlocking

println(s"Non-blocking operations took $timeNonBlocking ms")
println(s"Non-blocking results: $nonBlockingResult1, $nonBlockingResult2")

// Proper non-blocking handling using callbacks
combinedFuture.onComplete {
  case Success((r1, r2)) => println(s"Successfully completed with results: $r1, $r2")
  case Failure(e) => println(s"Operations failed with error: ${e.getMessage}")
}

// Assertions
assert(result1 == 2, "Blocking operation 1 failed")
assert(result2 == 4, "Blocking operation 2 failed")
assert(timeBlocking >= 2000, "Blocking operations should take at least 2 seconds")
assert(timeNonBlocking < timeBlocking, "Non-blocking should be faster than blocking")

Starting blocking operation 1
Finished blocking operation 1
Starting blocking operation 2
Finished blocking operation 1
Starting blocking operation 2
Finished blocking operation 2
Blocking operations took 2007 ms
Blocking results: 2, 4
Starting non-blocking operation 1
Starting non-blocking operation 2
Finished blocking operation 2
Blocking operations took 2007 ms
Blocking results: 2, 4
Starting non-blocking operation 1
Starting non-blocking operation 2
Finished non-blocking operation 2
Finished non-blocking operation 1
Non-blocking operations took 1010 ms
Non-blocking results: 2, 4
Successfully completed with results: 2, 4
Finished non-blocking operation 2
Finished non-blocking operation 1
Non-blocking operations took 1010 ms
Non-blocking results: 2, 4
Successfully completed with results: 2, 4


import scala.concurrent.{Future, Promise, Await}

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.duration._

import scala.util.{Success, Failure}

// Simulating a long-running operation

defined function simulateBlockingOperation
defined function simulateNonBlockingOperation
startTimeBlocking: Long = 1762756729257L
result1: Int = 2
result2: Int = 4
timeBlocking: Long = 2007L
startTimeNonBlocking: Long = 1762756731264L
future1: Future[Int] = Success(value = 2)
future2: Future[Int] = Success(value = 4)
combinedFuture: Future[(Int, Int)] = Success(value = (2, 4))
nonBlockingResult1: Int = 2
nonBlockingResult2: Int = 4
timeNonBlocking: Long = 1010L

## Exception Handling

Today I'm learning about different ways to handle errors in Scala. Starting with the basics:

### 9.1 Traditional Try-Catch Blocks

Let's start with traditional exception handling using try-catch blocks:

In [ ]:
// First attempt - basic division function
def divide(a: Int, b: Int) = a / b

// Oops! This crashes with divide(10, 0)
// Let's fix it with try-catch

def safeDivide(a: Int, b: Int) = {
  try {
    a / b
  } catch {
    case e: ArithmeticException => 
      println(s"Oops! Can't divide by zero!")
      0  // Return 0 instead of crashing
  }
}

// Test it
println(safeDivide(10, 2))  // Should work normally
println(safeDivide(10, 0))  // Should handle error

// Let's try something more interesting
def divideWithRetry(a: Int, b: Int, attempts: Int = 3): Int = {
  try {
    println(s"Attempting division $a / $b")
    a / b
  } catch {
    case e: ArithmeticException if attempts > 1 => 
      println(s"Failed, retrying... ${attempts-1} attempts left")
      divideWithRetry(a, b, attempts - 1)
    case e: ArithmeticException =>
      println("All attempts failed!")
      0
  }
}

// Let's see how it works
println("\nTrying with retries:")
divideWithRetry(10, 0)

Testing divide(10, 2):
Division operation completed

Testing divide(10, 0):
Cannot divide by zero!
Division operation completed
Division operation completed

Testing divide(10, 0):
Cannot divide by zero!
Division operation completed


defined function divide

### 9.2 Using scala.util.Try

`Try` is a type that represents a computation that may either result in an exception or return a successfully computed value. It's a more functional way to handle exceptions:

In [9]:
import scala.util.{Try, Success, Failure}

def divideWithTry(a: Int, b: Int): Try[Int] = Try(a / b)

// Test successful division
val successResult = divideWithTry(10, 2)
assert(successResult.isSuccess)
assert(successResult.get == 5)

// Test division by zero
val failureResult = divideWithTry(10, 0)
assert(failureResult.isFailure)
assert(failureResult.failed.get.isInstanceOf[ArithmeticException])

// Using map and getOrElse with Try
val mappedResult = divideWithTry(10, 2).map(_ * 2)
assert(mappedResult.getOrElse(0) == 10)

// Using pattern matching with Try
val resultDescription = divideWithTry(10, 2) match {
  case Success(result) => s"Success: $result"
  case Failure(ex) => s"Failed: ${ex.getMessage}"
}
assert(resultDescription == "Success: 5")

import scala.util.{Try, Success, Failure}


defined function divideWithTry
successResult: Try[Int] = Success(value = 5)
failureResult: Try[Int] = Failure(
  exception = java.lang.ArithmeticException: / by zero
)
mappedResult: Try[Int] = Success(value = 10)
resultDescription: String = "Success: 5"

### 9.4 Custom Exceptions

In Scala, we can create custom exceptions by extending the `Exception` class. Here's an example:

In [11]:
// Custom exception class
case class InvalidAgeException(message: String) extends Exception(message)

// Class using custom exception
class Person(age: Int) {
  if (age < 0) throw InvalidAgeException("Age cannot be negative")
  if (age > 150) throw InvalidAgeException("Age seems unrealistic")
  
  def getAge: Int = age
}

// Test valid age
try {
  val person1 = new Person(25)
  assert(person1.getAge == 25)
  println("Valid age case passed")
} catch {
  case e: InvalidAgeException => 
    println(s"Unexpected error: ${e.getMessage}")
    assert(false)
}

// Test invalid age
try {
  val person2 = new Person(-5)
  assert(false) // Should not reach this line
} catch {
  case e: InvalidAgeException => 
    assert(e.getMessage == "Age cannot be negative")
    println("Invalid age case (negative) caught successfully")
}

// Test unrealistic age
try {
  val person3 = new Person(200)
  assert(false) // Should not reach this line
} catch {
  case e: InvalidAgeException => 
    assert(e.getMessage == "Age seems unrealistic")
    println("Invalid age case (unrealistic) caught successfully")
}

Valid age case passed
Invalid age case (negative) caught successfully
Invalid age case (unrealistic) caught successfully
Invalid age case (negative) caught successfully
Invalid age case (unrealistic) caught successfully


defined class InvalidAgeException
defined class Person

## 10. Futures

Futures in Scala provide a way to perform computations asynchronously. They are especially useful for handling operations that might take a long time to complete, such as network calls or complex computations.

In [ ]:
import scala.concurrent.{Future, Promise}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.util.{Success, Failure}
import scala.concurrent.duration._

// Simple Future example
def calculateFactorial(n: Int): Future[BigInt] = Future {
  println(s"Calculating factorial of $n...")
  Thread.sleep(1000) // Simulate long computation
  (1 to n).foldLeft(BigInt(1))(_ * _)
}

// Test the Future
val futureResult = calculateFactorial(5)
futureResult.onComplete {
  case Success(result) => println(s"Factorial result: $result")
  case Failure(e) => println(s"Calculation failed: ${e.getMessage}")
}

// Using map and flatMap with Futures
val doubledFactorial = calculateFactorial(5).map(_ * 2)
doubledFactorial.foreach(result => println(s"Doubled factorial: $result"))

// Combining multiple Futures
val fut1 = calculateFactorial(3)
val fut2 = calculateFactorial(4)

val combinedFuture = for {
  result1 <- fut1
  result2 <- fut2
} yield (result1, result2)

combinedFuture.foreach { case (r1, r2) =>
  println(s"Combined results: $r1 and $r2")
}

// Error handling with Futures
def riskyCalculation(n: Int): Future[Int] = Future {
  if (n < 0) throw new IllegalArgumentException("Number must be positive")
  n * 2
}

val successfulFuture = riskyCalculation(5)
val failedFuture = riskyCalculation(-1)

successfulFuture.foreach(println)
failedFuture.recover {
  case e: IllegalArgumentException => println(s"Recovered from error: ${e.getMessage}")
}

// Using Promise for manual completion
val promise = Promise[String]()
val future = promise.future

// Complete the promise after a delay
Future {
  Thread.sleep(1000)
  promise.success("Promise fulfilled!")
}

future.foreach(println)

// Wait for results to verify (only for demonstration)
import scala.concurrent.Await
val results = Await.result(combinedFuture, 5.seconds)
assert(results._1 == 6, "Factorial of 3 should be 6")
assert(results._2 == 24, "Factorial of 4 should be 24")

val successResult = Await.result(successfulFuture, 1.second)
assert(successResult == 10, "Successful calculation should double 5 to 10")

### 10.1 Future Patterns

Let's explore some common patterns when working with Futures:

In [ ]:
// Pattern 1: Future sequence
def fetchUserData(id: Int): Future[String] = Future {
  Thread.sleep(100) // Simulate network delay
  s"User$id"
}

// Sequential vs Parallel execution
val userIds = List(1, 2, 3, 4, 5)

// Sequential (one after another)
val sequentialFutures = userIds.foldLeft(Future.successful(List.empty[String])) { 
  (acc, id) => 
    for {
      list <- acc
      user <- fetchUserData(id)
    } yield list :+ user
}

// Parallel (all at once)
val parallelFutures = Future.sequence(userIds.map(fetchUserData))

// Pattern 2: Future firstCompletedOf
val futures = List(
  Future { Thread.sleep(1000); "Slow" },
  Future { Thread.sleep(500); "Medium" },
  Future { Thread.sleep(100); "Fast" }
)

val fastest = Future.firstCompletedOf(futures)
fastest.foreach(result => println(s"Fastest result: $result"))

// Pattern 3: Fallback mechanism
def primaryOperation: Future[String] = Future {
  throw new Exception("Primary operation failed")
}

def fallbackOperation: Future[String] = Future {
  "Fallback result"
}

val withFallback = primaryOperation.fallbackTo(fallbackOperation)
withFallback.foreach(println)

// Wait for results to verify (only for demonstration)
val parallelResults = Await.result(parallelFutures, 2.seconds)
assert(parallelResults.size == 5, "Should have 5 user results")
assert(parallelResults.head == "User1", "First user should be User1")

val fallbackResult = Await.result(withFallback, 1.second)
assert(fallbackResult == "Fallback result", "Should get fallback result when primary fails")